In [2]:
# from pyspark import 

In [7]:
inputDF = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("datathon_tadata.csv")

In [10]:
inputDF.createOrReplaceTempView("input_data")

In [13]:
sqlContext.sql("SELECT user_id FROM input_data").show()

+-------+
|user_id|
+-------+
|      1|
|      2|
|      3|
|      4|
|      5|
|      6|
|      7|
|      8|
|      9|
|     10|
|     11|
|     12|
|     13|
|     14|
|     15|
|     16|
|     17|
|     18|
|     19|
|     20|
+-------+
only showing top 20 rows



In [15]:
inputDF.rdd.take(1)

[Row(user_id=1, day=datetime.datetime(2017, 1, 10, 0, 0), gender=1, p_sessionActivity=1, p_AddToCart=0, p_trafficChannel='O', p_sessionDuration=73, p_pageViews=1, daysToCheckin='NA', osType=8, osTypeName='osx', daysFromPreviousVisit=10, p_TotalPrice='NA', isExclusiveMember=0, loggedIn=0, p_MapInteraction=0, BookingPurchase=0)]

In [1]:
sc